In [4]:
import numpy as np
import matplotlib.pyplot as plt

import random

## Задача 0 (Свахи Круглого Стола)
> К вам в гости пришли $n$ мужчин и $n$ девушек.
> Вы решаете их посадит за круглый стол.
>
> С какой вероятностью рядом с каждым мужчиной будет сидеть ровно одна женщина, а рядом с каждой женщиной -- мужчина?

Мы в этой задаче на паре забыли рассмотреть рассадки (м, ж, ж, м) и (ж, м, м, ж). Поэтому вероятность в 2 раза выше

In [56]:
from math import comb, factorial

def monte_carlo_round_table(n, trials=500_000):
  total = 2 * n
  success = 0
  for _ in range(trials):
    "0 = мужчина, 1 = женщина"
    seats = [0] * n + [1] * n
    random.shuffle(seats)
    "Проверяем: у каждого мужчины оба соседа — женщины?"
    ok = True
    for i in range(total):
      if seats[i] == 0:  # мужчина
        left = seats[i - 1]
        right = seats[(i + 1) % total]
        if (left == right):
          ok = False
          break
      elif seats[i] == 1:  # жегщина
        left = seats[i - 1]
        right = seats[(i + 1) % total]
        if (left == right):
          ok = False
          break
    if ok:
      success += 1
  return success / trials

def theoretical_probability(n):
  "P = 4 / C(2n, n)"
  return 4 / comb(2 * n, n) * ((n + 1) % 2)

n = 6

prob_mc = monte_carlo_round_table(n)
prob_theor = theoretical_probability(n)

print(f"Теоретическая вероятность: {prob_theor:.6f}")
print(f"Оценка Монте-Карло:        {prob_mc:.6f}")
print(f"Абсолютная ошибка:         {abs(prob_mc - prob_theor):.6f}")


Теоретическая вероятность: 0.004329
Оценка Монте-Карло:        0.004402
Абсолютная ошибка:         0.000073


# Геометрическая вероятность

> Рассмотрим числовую прямую и отрезок $[a, b] = \Omega$. Пусть случайным образом выбирается точка из этого отрезка. Это означает, что любая точка отрезка $[a, b]$ может появиться в результате эксперимента с равными шансами.
>
> Зададим вероятностную меру для любого множества $A \in \mathcal{F}_{[a,b]}$, где $\mathcal{F}_{[a,b]}$ — множество подмножеств отрезка $[a,b]$, для которых можно определить "длину". Вероятностная мера, называемая геометрической вероятностью, может быть определена следующим образом:
>
>\begin{equation*}
P(A) = \frac{\mu(A)}{\mu[a,b]} = \frac{\mu(A)}{b - a},
\end{equation*}
>
> где через $\mu(A)$ обозначена "длина" множества $A$.


> На плоскости у множества можно определить площадь. А в трёхмерном пространстве -- объём. Множества, для которых можно корректно ввести такие понятия назовём *измеримыми*.

> В более общем случае геометрическая вероятность определяется аналогично. В качестве множества элементарных событий рассмотрим некоторую измеримую область $\Omega \subset \mathbb{R}^k$. В качестве множества событий нельзя рассматривать множество всех подмножеств множества $\Omega$, так как не все подмножества множества $\Omega$ измеримы. Поэтому за события мы будем принимать только измеримые множества. Через $\mu(\Omega)$ обозначим меру в $\mathbb{R}^k$ множества $\Omega$. Используя принцип геометрической вероятности по отношению к мере $\mu$, можно определить вероятностную меру для любого измеримого множества $A \subset \Omega \subset \mathbb{R}^k$ следующим образом:
>
> \begin{equation*}
P(A) = \frac{\mu(A)}{\mu(\Omega)}.
\end{equation*}
>
> Последнее равенство принято называть определением геометрической вероятности.

#### Далее идёт технический пример: вероятность того, что иголка, бросаемая в квадрат, попадёт в круг заданного радиуса в центре квадрата. Подумайте, как вычислить теоретическую вероятность?

In [11]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

# Параметры
radius = 1.0          # радиус круга
side = 2.0             # сторона квадрата (от -1 до 1)
total_frames = 100     # число кадров
points_per_frame = 50  # точек, добавляемых на каждом кадре
total_points = total_frames * points_per_frame

# Генерация случайных точек заранее (для воспроизводимости)
np.random.seed(42)
x = np.random.uniform(-1, 1, total_points)
y = np.random.uniform(-1, 1, total_points)
# Проверка попадания в круг
inside = x**2 + y**2 <= radius**2

# Создание фигуры и осей
fig, (ax_main, ax_prob) = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Геометрическая вероятность: попадание точки в круг', fontsize=14)

# Левый график: квадрат и круг
ax_main.set_xlim(-1.1, 1.1)
ax_main.set_ylim(-1.1, 1.1)
ax_main.set_aspect('equal')
ax_main.grid(True, linestyle='--', alpha=0.7)
ax_main.set_title('Случайные точки')
# Рисуем квадрат
square = plt.Rectangle((-1, -1), 2, 2, fill=False, edgecolor='black', linewidth=2)
ax_main.add_patch(square)
# Рисуем круг
circle = plt.Circle((0, 0), radius, color='red', fill=False, linewidth=2)
ax_main.add_patch(circle)

# Инициализируем пустые объекты для точек (разбросанные и внутри)
scat_out = ax_main.scatter([], [], s=10, color='blue', alpha=0.5, label='Вне круга')
scat_in = ax_main.scatter([], [], s=10, color='red', alpha=0.5, label='Внутри круга')
ax_main.legend(loc='upper right')

# Правый график: эволюция вероятности
ax_prob.set_xlim(0, total_points)
ax_prob.set_ylim(0.5, 1.0)
ax_prob.axhline(y=np.pi/4, color='red', linestyle='--', label=f'Теоретическая вероятность π/4 ≈ {np.pi/4:.4f}')
ax_prob.set_xlabel('Количество точек')
ax_prob.set_ylabel('Оценка вероятности')
ax_prob.set_title('Сходимость оценки')
ax_prob.grid(True, linestyle='--', alpha=0.7)
ax_prob.legend(loc='lower right')

# Линия для отображения текущей вероятности
line_prob, = ax_prob.plot([], [], 'b-', linewidth=2)
# Текущая вероятность в виде текста
text_prob = ax_prob.text(0.02, 0.95, '', transform=ax_prob.transAxes, fontsize=12,
                         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Данные для анимации
def init():
    scat_out.set_offsets(np.empty((0, 2)))
    scat_in.set_offsets(np.empty((0, 2)))
    line_prob.set_data([], [])
    text_prob.set_text('')
    return scat_out, scat_in, line_prob, text_prob

def update(frame):
    # Текущее количество точек: все точки до этого кадра
    n_points = (frame + 1) * points_per_frame
    x_curr = x[:n_points]
    y_curr = y[:n_points]
    inside_curr = inside[:n_points]

    # Разделяем точки
    x_in = x_curr[inside_curr]
    y_in = y_curr[inside_curr]
    x_out = x_curr[~inside_curr]
    y_out = y_curr[~inside_curr]

    # Обновляем scatter
    if len(x_in) > 0:
        scat_in.set_offsets(np.c_[x_in, y_in])
    else:
        scat_in.set_offsets(np.empty((0, 2)))

    if len(x_out) > 0:
        scat_out.set_offsets(np.c_[x_out, y_out])
    else:
        scat_out.set_offsets(np.empty((0, 2)))

    # Оценка вероятности (доля попаданий)
    prob = np.mean(inside_curr) if n_points > 0 else 0
    # Обновляем график вероятности
    x_data = np.arange(1, n_points+1)
    y_data = np.cumsum(inside[:n_points]) / np.arange(1, n_points+1)
    line_prob.set_data(x_data, y_data)

    # Текст с текущими значениями
    text_prob.set_text(f'N = {n_points}\nP ≈ {prob:.4f}\nπ/4 = {np.pi/4:.4f}')

    return scat_out, scat_in, line_prob, text_prob

# Создаем анимацию
anim = FuncAnimation(fig, update, frames=total_frames, init_func=init,
                     blit=True, repeat=True, interval=100)

# Для отображения в Colab используем HTML5 video
plt.close(fig)  # предотвращаем дублирование статического графика
display(HTML(anim.to_html5_video()))

## Задача 1 (Сигнал)

> В интервале времени $[0, T]$ в случайный момент $u$ появляется сигнал длительности $\Delta$.
> Приёмник включается в случайный момент $v \in [0, T]$ на время $t$.
> Предположим, что точка $(u, v)$ распределена равномерно на квадрате $[0, T]^2$
>
> Какова вероятность обнаружения сигнала?

In [61]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

def detection_condition(u, v, Delta, t):
    "Проверка, пересекаются ли интервалы [u, u+Delta] и [v, v+t]."
    return not (v > u + Delta or u > v + t)

def exact_probability(T, Delta, t):
    if Delta <= 0 or t <= 0:
        return 0.0
    area = (T * T) - (T - t)**2 / 2 - (T - Delta)**2 / 2
    return area / (T * T)

def simulate(T, Delta, t, n_trials=10**6, seed=42):
    np.random.seed(seed)
    u = np.random.uniform(0, T, n_trials)
    v = np.random.uniform(0, T, n_trials)

    detected = (u <= v + t) & (v <= u + Delta)
    prob_estimate = np.mean(detected)

    prob_exact = exact_probability(T, Delta, t)
    return prob_estimate, prob_exact

T = 12.0
Delta = 3.0
t = 3.0
N = 1_000_000

print(f"Параметры: T = {T}, Δ = {Delta}, t = {t}")

prob_est, prob_ex = simulate(T, Delta, t, N)
print(f"Оценка вероятности (Монте-Карло): {prob_est:.6f}")
print(f"Точная вероятность:               {prob_ex:.6f}")
print(f"Абсолютная ошибка:                {abs(prob_est - prob_ex):.6f}")

Параметры: T = 12.0, Δ = 3.0, t = 3.0
Оценка вероятности (Монте-Карло): 0.438406
Точная вероятность:               0.437500
Абсолютная ошибка:                0.000906


## Задача 2 (Трамвай)

> Пассажир может воспользоваться трамваями двух маршрутов, следующих с интервалами $T_1$, $T_2$.  
> Момент прихода пассажира определяет на отрезках $[0, T_1]$, $[0, T_2]$ числа $u$ и $v$, равные временам, оставшимся до прихода трамвая соответствующего маршрута.  
>
> Предполагая, что точка $(u, v)$ равномерно распределена на $\Omega = \{(u, v): 0 \leq u \leq T_1,\ 0 \leq v \leq T_2\}$, найти вероятность того, что пассажир, пришедший на остановку, будет ждать не дольше $t$ ($0 < t < \min(T_1, T_2)$).

In [62]:
def monte_carlo_waiting_probability(T1, T2, t, num_simulations=1_000_000):
    "Генерируем u и v из равномерных распределений"
    u = np.random.uniform(0, T1, num_simulations)
    v = np.random.uniform(0, T2, num_simulations)
    "Время ожидания – минимум из u и v"
    wait_time = np.minimum(u, v)

    "Оценка вероятности"
    prob_estimate = np.mean(wait_time <= t)

    return prob_estimate

def theoretical_probability(T1, T2, t):
    "Аналитическая вероятность для случая 0 < t < min(T1,T2)"
    return 1 - ((T1 - t) / T1) * ((T2 - t) / T2)


T1, T2 = 10.0, 15.0
t = 5.0

prob_mc = monte_carlo_waiting_probability(T1, T2, t, num_simulations=2_000_000)
prob_theor = theoretical_probability(T1, T2, t)

print(f"Параметры: T1 = {T1}, T2 = {T2}, t = {t}")
print(f"Теоретическая вероятность: {prob_theor:.6f}")
print(f"Оценка Монте-Карло:        {prob_mc:.6f}")
print(f"Абсолютная ошибка:         {abs(prob_mc - prob_theor):.6f}")

Параметры: T1 = 10.0, T2 = 15.0, t = 5.0
Теоретическая вероятность: 0.666667
Оценка Монте-Карло:        0.666708
Абсолютная ошибка:         0.000041


## Задача 3 (Монетка и паркет)

> На паркет, составленный из правильных $k$-угольников со стороной $a$, случайно бросается монета радиуса $r$.
> Найти вероятность того, что упавшая монета не задевает границу ни одного из $k$-угольников паркета для $k = 4$.

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle
from IPython.display import HTML, display

# Параметры задачи
a = 3.0          # сторона квадрата паркета
r = 0.6          # радиус монеты
inner = a - 2*r  # сторона внутренней безопасной области
if inner <= 0:
    raise ValueError("Монета слишком большая, вероятность 0")

# Область отображения: 3x3 квадрата
x_min, x_max = -3, 3
y_min, y_max = -3, 3
n_squares = 3    # количество квадратов по каждой оси

# Вычисляем координаты безопасных зон (внутренних квадратов)
safe_rects = []
for i in range(n_squares):
    for j in range(n_squares):
        left = x_min + i * a
        bottom = y_min + j * a
        safe_rects.append((left + r, bottom + r, left + a - r, bottom + a - r))

# Генерация случайных точек (центров монет)
total_frames = 100
points_per_frame = 50
total_points = total_frames * points_per_frame
np.random.seed(42)
x = np.random.uniform(x_min, x_max, total_points)
y = np.random.uniform(y_min, y_max, total_points)

# Проверка, попадает ли точка в безопасную зону
def is_safe(x, y):
    for (l, b, rgt, t) in safe_rects:
        if l <= x <= rgt and b <= y <= t:
            return True
    return False

inside = np.array([is_safe(x[i], y[i]) for i in range(total_points)])

# Создание фигуры
fig, (ax_main, ax_prob) = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('Монета на паркете из квадратов', fontsize=14)

# Левый график: паркет и точки
ax_main.set_xlim(x_min, x_max)
ax_main.set_ylim(y_min, y_max)
ax_main.set_aspect('equal')
ax_main.grid(True, linestyle='--', alpha=0.5)
ax_main.set_title('Центры монет: красные – не задевают границы, синие – задевают')

# Рисуем сетку квадратов
for i in range(n_squares + 1):
    ax_main.axvline(x_min + i * a, color='black', linewidth=1)
    ax_main.axhline(y_min + i * a, color='black', linewidth=1)

# Закрашиваем безопасные области
for (l, b, rgt, t) in safe_rects:
    rect = Rectangle((l, b), rgt - l, t - b, facecolor='lightgreen', alpha=0.5, edgecolor='none')
    ax_main.add_patch(rect)

# Объекты для точек
scat_safe = ax_main.scatter([], [], s=10, color='red', alpha=0.6, label='Безопасно')
scat_unsafe = ax_main.scatter([], [], s=10, color='blue', alpha=0.6, label='Задевает границу')
ax_main.legend(loc='upper right')

# Правый график: сходимость вероятности
theor_prob = (inner / a) ** 2
ax_prob.set_xlim(0, total_points)
ax_prob.set_ylim(0, 1.0)
ax_prob.axhline(y=theor_prob, color='red', linestyle='--', label=f'Теоретическая: {theor_prob:.4f}')
ax_prob.set_xlabel('Количество точек')
ax_prob.set_ylabel('Оценка вероятности')
ax_prob.set_title('Сходимость оценки')
ax_prob.grid(True, linestyle='--', alpha=0.7)
ax_prob.legend(loc='lower right')

line_prob, = ax_prob.plot([], [], 'b-', linewidth=2)
text_prob = ax_prob.text(0.02, 0.95, '', transform=ax_prob.transAxes, fontsize=12,
                         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Функции анимации
def init():
    scat_safe.set_offsets(np.empty((0, 2)))
    scat_unsafe.set_offsets(np.empty((0, 2)))
    line_prob.set_data([], [])
    text_prob.set_text('')
    return scat_safe, scat_unsafe, line_prob, text_prob

def update(frame):
    n_points = (frame + 1) * points_per_frame
    x_curr = x[:n_points]
    y_curr = y[:n_points]
    inside_curr = inside[:n_points]

    # Разделяем точки
    x_safe = x_curr[inside_curr]
    y_safe = y_curr[inside_curr]
    x_unsafe = x_curr[~inside_curr]
    y_unsafe = y_curr[~inside_curr]

    if len(x_safe) > 0:
        scat_safe.set_offsets(np.c_[x_safe, y_safe])
    if len(x_unsafe) > 0:
        scat_unsafe.set_offsets(np.c_[x_unsafe, y_unsafe])

    # Оценка вероятности
    prob = np.mean(inside_curr) if n_points > 0 else 0
    x_data = np.arange(1, n_points + 1)
    y_data = np.cumsum(inside[:n_points]) / np.arange(1, n_points + 1)
    line_prob.set_data(x_data, y_data)

    text_prob.set_text(f'N = {n_points}\nP ≈ {prob:.4f}\nТеор = {theor_prob:.4f}')

    return scat_safe, scat_unsafe, line_prob, text_prob

# Запуск анимации
anim = FuncAnimation(fig, update, frames=total_frames, init_func=init,
                     blit=True, repeat=True, interval=100)
plt.close(fig)
display(HTML(anim.to_html5_video()))

## Задача 4 (Задача Бертрана)
> Рассматривается окружность радиуса $R$, в которую вписан равносторонний треугольник.
> Случайным образом в данной окружности выбирается хорда.
>
> Какова вероятность того, что длина хорды окажется больше стороны равностороннего треугольника, вписанного в эту окружность?

Мы не успели посмотреть её на паре. Обсуждение можете посмотреть на стр. 21, $\S 5$, Гл. 1 в учебнике "Теория Вероятностей и Математическая Статистика", В. М. Буре, Е. М. Парилиной.

Обратите внимание на то, что ответ зависит от трактовки и выбранной модели!

# Деп

> Это игра для двух игроков. Игроки играют друг против друга.
>
> Каждый ($i$-ый) игрок делает следующее: выбирает место, где он встанет ($p_i$) и куда будет стрелять ($t_i$).
> Место идентифицируется числом от 1 до 1000.
>
> Цель: не быть убитым и попасть рядом с местом, где стоит противник.
>
> Считается, что игрок $i$ убит, если выстрел $j$-ого игрока попал рядом: $|p_i - t_j| < 500$.
> Матрица выигрыша следующая:
>
> |             | Попал    | Промахнулся |
> | :---------  | :------: | ----:       |
> | Попал       |   -1/-1  | 0/2         |
> | Промахнулся |   2/0    | 1/1         |
>